In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import numpy as np
import pandas as pd
from datetime import datetime
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.impute import KNNImputer
from scipy.cluster.hierarchy import linkage, fcluster, dendrogram
from sklearn.cluster import DBSCAN, HDBSCAN
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from scipy.stats import kendalltau, norm
import cvxpy as cp
import calendar
from datetime import date
import pickle

In [3]:
folder_path = '/content/drive/My Drive/fednlp/ai/'
prices_info = pd.read_csv(f'{folder_path}prices_info.csv.gz', compression='gzip')
ratios_info = pd.read_csv(f'{folder_path}ratios.csv.gz', compression='gzip')
sector_info = pd.read_csv(f'{folder_path}sector_info.csv.gz', compression='gzip')
sp500 = pd.read_pickle(f'{folder_path}sp500.pkl')
relations = pd.read_csv(f'{folder_path}lorlxamlga34g2og.csv')

In [ ]:
sector_info2 = sector_info[['tic','datadate','gsubind']].drop_duplicates().dropna()
sector_info2['gsubind'] = sector_info2['gsubind'].apply(lambda x: str(int(x)))
sector_info2['sector'] = sector_info2['gsubind'].apply(lambda x: x[:4])
sector_info2['subsector'] = sector_info2['gsubind'].apply(lambda x: x[:6])

sector_info2['datadate'] = pd.to_datetime(sector_info2['datadate'])

sector_info2 = sector_info2.sort_values(['tic', 'datadate'])

sector_info2['grp'] = ((sector_info2['tic'] != sector_info2['tic'].shift()) |
                      (sector_info2['gsubind'] != sector_info2['gsubind'].shift())).cumsum()

intervals = sector_info2.groupby(['tic', 'gsubind', 'grp'])['datadate'].agg(start_='min', end_='max').reset_index()

merged = pd.merge(intervals, intervals, on='gsubind', suffixes=('_src', '_tgt'))
merged = merged[merged['tic_src'] != merged['tic_tgt']]

merged['overlap_start'] = np.maximum(merged['start__src'], merged['start__tgt'])
merged['overlap_end'] = np.minimum(merged['end__src'], merged['end__tgt'])

merged = merged[merged['overlap_start'] <= merged['overlap_end']]

final_df = merged.groupby(['tic_src', 'tic_tgt', 'gsubind'], as_index=False).agg(
    start_=('overlap_start', 'min'),
    end_=('overlap_end', 'max')
)

del merged

final_df.rename(columns={'tic_src': 'source_ticker', 'tic_tgt': 'target_ticker'}, inplace=True)
ind_pairs = final_df[['source_ticker', 'target_ticker', 'start_', 'end_', 'gsubind']]

del final_df

In [ ]:
sector_info2['grp'] = ((sector_info2['tic'] != sector_info2['tic'].shift()) |
                      (sector_info2['subsector'] != sector_info2['subsector'].shift())).cumsum()

intervals = sector_info2.groupby(['tic', 'subsector', 'grp'])['datadate'].agg(start_='min', end_='max').reset_index()

merged = pd.merge(intervals, intervals, on='subsector', suffixes=('_src', '_tgt'))
merged = merged[merged['tic_src'] != merged['tic_tgt']]

merged['overlap_start'] = np.maximum(merged['start__src'], merged['start__tgt'])
merged['overlap_end'] = np.minimum(merged['end__src'], merged['end__tgt'])

merged = merged[merged['overlap_start'] <= merged['overlap_end']]

final_df = merged.groupby(['tic_src', 'tic_tgt', 'subsector'], as_index=False).agg(
    start_=('overlap_start', 'min'),
    end_=('overlap_end', 'max')
)

del merged

final_df.rename(columns={'tic_src': 'source_ticker', 'tic_tgt': 'target_ticker'}, inplace=True)
subsector_pairs = final_df[['source_ticker', 'target_ticker', 'start_', 'end_', 'subsector']]

del final_df

In [ ]:
sector_info2['grp'] = ((sector_info2['tic'] != sector_info2['tic'].shift()) |
                      (sector_info2['sector'] != sector_info2['sector'].shift())).cumsum()

intervals = sector_info2.groupby(['tic', 'sector', 'grp'])['datadate'].agg(start_='min', end_='max').reset_index()

merged = pd.merge(intervals, intervals, on='sector', suffixes=('_src', '_tgt'))
merged = merged[merged['tic_src'] != merged['tic_tgt']]

merged['overlap_start'] = np.maximum(merged['start__src'], merged['start__tgt'])
merged['overlap_end'] = np.minimum(merged['end__src'], merged['end__tgt'])

merged = merged[merged['overlap_start'] <= merged['overlap_end']]

final_df = merged.groupby(['tic_src', 'tic_tgt', 'sector'], as_index=False).agg(
    start_=('overlap_start', 'min'),
    end_=('overlap_end', 'max')
)

del merged

final_df.rename(columns={'tic_src': 'source_ticker', 'tic_tgt': 'target_ticker'}, inplace=True)
sector_pairs = final_df[['source_ticker', 'target_ticker', 'start_', 'end_', 'sector']]

del final_df

In [ ]:
sector_pairs.to_feather(f'{folder_path}sector_pairs.feather')
ind_pairs.to_feather(f'{folder_path}ind_pairs.feather')
subsector_pairs.to_feather(f'{folder_path}subsector_pairs.feather')

In [ ]:
rels2.to_feather(f'{folder_path}relsations.feather')

In [ ]:
sector_pairs['Val'] = 1
ind_pairs['Val'] = 1
subsector_pairs['Val'] = 1
comb_sector = pd.concat([sector_pairs[['source_ticker','target_ticker','start_','end_','Val']], ind_pairs[['source_ticker','target_ticker','start_','end_','Val']],
                         subsector_pairs[['source_ticker','target_ticker','start_','end_','Val']]])

In [ ]:
comb_sector = comb_sector.groupby(['source_ticker','target_ticker','start_','end_'])['Val'].sum().reset_index()

In [ ]:
comb_sector['start_'] = pd.to_datetime(comb_sector['start_'])
comb_sector['end_'] = pd.to_datetime(comb_sector['end_'])

In [ ]:
comb_sector['Val'].max()

3

In [ ]:
rels2 = relations[['source_ticker','target_ticker','start_','end_','rel_type']].drop_duplicates().dropna()
rel_strength = {'COMPETITOR':3, 'SUPPLIER':4, 'CUSTOMER':4}

rels2['Val'] = rels2['rel_type'].apply(lambda x: rel_strength[x])
rels2['start_'] = pd.to_datetime(rels2['start_'])
rels2['end_'] = rels2['end_'].apply(lambda x: x.replace('4000','2026'))
rels2['end_'] = pd.to_datetime(rels2['end_'])
rels2 = rels2.drop_duplicates()

In [ ]:
rels2[(rels2['source_ticker'] == 'TTWO') & (rels2['target_ticker'] == 'MSFT')]

,source_ticker,target_ticker,start_,end_,rel_type,Val
182175,TTWO,MSFT,2019-08-08,2026-01-01,COMPETITOR,3
568011,TTWO,MSFT,2023-11-15,2025-05-20,COMPETITOR,3
568012,TTWO,MSFT,2025-05-20,2025-05-20,COMPETITOR,3
1434911,TTWO,MSFT,2019-08-08,2020-06-11,CUSTOMER,4
1434912,TTWO,MSFT,2020-06-11,2021-05-19,CUSTOMER,4
1434913,TTWO,MSFT,2021-05-19,2021-05-19,CUSTOMER,4
1616468,TTWO,MSFT,2021-05-19,2022-11-16,CUSTOMER,4
1616470,TTWO,MSFT,2022-11-16,2022-11-16,CUSTOMER,4
1616471,TTWO,MSFT,2022-11-16,2023-10-13,CUSTOMER,4
1616472,TTWO,MSFT,2023-10-13,2023-10-16,CUSTOMER,4


In [ ]:
rels2[(rels2['source_ticker'] == 'WES') & (rels2['target_ticker'] == 'OXY')]

,source_ticker,target_ticker,start_,end_,Val
873534,WES,OXY,2019-08-14,2020-03-10,11
873535,WES,OXY,2020-03-11,2026-01-01,8


In [ ]:
bounds_start = rels2[['source_ticker', 'target_ticker', 'start_']].rename(columns={'start_': 'date'})
bounds_end = rels2[['source_ticker', 'target_ticker', 'end_']].copy()
bounds_end['date'] = bounds_end['end_'] + pd.Timedelta(days=1)

all_dates = pd.concat([bounds_start, bounds_end]).drop_duplicates()
all_dates = all_dates.sort_values(['source_ticker', 'target_ticker', 'date'])

# Determine interval start and end (next date - 1 day)
all_dates['next_date'] = all_dates.groupby(['source_ticker', 'target_ticker'])['date'].shift(-1)
atomic = all_dates.dropna(subset=['next_date']).copy()
atomic['end_'] = atomic['next_date'] - pd.Timedelta(days=1)
atomic = atomic.rename(columns={'date': 'start_'})
atomic = atomic[['source_ticker', 'target_ticker', 'start_', 'end_']]
# Filter out invalid intervals (gaps)
atomic = atomic[atomic['start_'] <= atomic['end_']]

# 3. Map atomic intervals to original data
# Merge on Tickers only to create a grid, then filter by date range
merged = pd.merge(atomic, rels2, on=['source_ticker', 'target_ticker'], suffixes=('', '_orig'))

# Keep rows where the atomic interval falls within the original validity period
merged = merged[
    (merged['start_'] >= merged['start__orig']) &
    (merged['end_'] <= merged['end__orig'])
]

# 4. Apply Logic:
# A. Max Val for same relationship type within an interval
step1 = merged.groupby(['source_ticker', 'target_ticker', 'start_', 'end_', 'rel_type'])['Val'].max().reset_index()

# B. Sum Val across different relationship types within that same interval
step2 = step1.groupby(['source_ticker', 'target_ticker', 'start_', 'end_'])['Val'].sum().reset_index()

# 5. Merge consecutive intervals with identical values
step2 = step2.sort_values(['source_ticker', 'target_ticker', 'start_'])
g = step2.groupby(['source_ticker', 'target_ticker'])

# Calculate grouping key: increment if Val changes or if there is a time gap
step2['prev_end'] = g['end_'].shift(1)
step2['prev_val'] = g['Val'].shift(1)
step2['grp_change'] = (
    (step2['start_'] != (step2['prev_end'] + pd.Timedelta(days=1))) |
    (step2['Val'] != step2['prev_val'])
)
step2['grp'] = step2.groupby(['source_ticker', 'target_ticker'])['grp_change'].cumsum()

final_df = step2.groupby(['source_ticker', 'target_ticker', 'grp', 'Val']).agg(
    start_=('start_', 'min'),
    end_=('end_', 'max')
).reset_index()

rels2 = final_df[['source_ticker', 'target_ticker', 'start_', 'end_', 'Val']]

In [ ]:
rels2['Val'].max()

11

In [ ]:
rels2

,source_ticker,target_ticker,start_,end_,Val
0,000004,002099,2016-01-04,2018-01-25,4
1,000004,002419,2022-12-13,2026-01-01,4
2,000004,2318,2021-09-20,2026-01-01,4
3,000004,301093,2018-02-09,2018-09-24,4
4,000004,3333,2021-09-20,2025-05-29,4
...,...,...,...,...,...
894376,ZZZ,PRPL,2021-03-22,2024-06-06,4
894377,ZZZ,SCC,2015-12-21,2019-05-15,3
894378,ZZZ,SGI,2015-12-21,2024-06-06,4
894379,ZZZ,SRTA.XX1,2018-06-01,2026-01-01,4


In [ ]:
comb_sector['Val'].max()

3

In [ ]:
comb_sector = pd.concat([comb_sector, rels2[['source_ticker','target_ticker','start_','end_','Val']]])

In [ ]:
comb_sector = comb_sector.groupby(['source_ticker','target_ticker','start_','end_'])['Val'].sum().reset_index()

In [ ]:
comb_sector['Val'].max()

22

In [ ]:
comb_sector[comb_sector['Val'] == 22]

,source_ticker,target_ticker,start_,end_,Val
76872,017480,004020,2023-05-22,2026-01-01,22
76873,017480,005490,2023-05-22,2026-01-01,22
103166,053260,005490,2023-06-06,2026-01-01,22
142027,1230,2319,2016-07-29,2017-10-10,22
147735,135,857,2019-06-21,2019-06-25,22
...,...,...,...,...,...
46618565,VZIO,WBD,2024-02-29,2026-01-01,22
46904191,WES,OXY,2019-08-14,2020-03-10,22
46908673,WES.XX1,OXY,2019-08-14,2020-08-17,22
47403329,WOW,CMCSA,2024-12-17,2026-01-01,22


In [ ]:
comb_sector2 = comb_sector[comb_sector['source_ticker'].isin(sp500['ticker'])].copy()
comb_sector2 = comb_sector2[comb_sector2['target_ticker'].isin(sp500['ticker'])]
comb_sector2 = comb_sector2[comb_sector2['Val'] > 1].reset_index(drop=True)
comb_sector2.to_feather('econ_link_filt.feather')

In [ ]:
comb_sector2.columns

Index(['source_ticker', 'target_ticker', 'start_', 'end_', 'Val'], dtype='object')

In [ ]:
# 1. Standardize Dates (Remove time components) and Ensure Numeric Values
comb_sector2['start_'] = pd.to_datetime(comb_sector2['start_']).dt.normalize()
comb_sector2['end_'] = pd.to_datetime(comb_sector2['end_']).dt.normalize()
comb_sector2['Val'] = pd.to_numeric(comb_sector2['Val'])

# 2. Decompose into Start and End Events
# Start event: Value is added on this day
starts = comb_sector2[['source_ticker', 'target_ticker', 'start_', 'Val']].copy()
starts.columns = ['source_ticker', 'target_ticker', 'date', 'delta']

# End event: Value is removed on the day AFTER the end date
ends = comb_sector2[['source_ticker', 'target_ticker', 'end_', 'Val']].copy()
ends['date'] = ends['end_'] + pd.Timedelta(days=1)
ends['delta'] = -ends['Val']
ends = ends[['source_ticker', 'target_ticker', 'date', 'delta']]

# 3. Combine and Aggregate by Date
# CRITICAL STEP: Sum deltas occurring on the exact same date for the same pair
# This prevents processing 'Start' and 'End' events separately for the same day
events = pd.concat([starts, ends])
events = events.groupby(['source_ticker', 'target_ticker', 'date'], as_index=False)['delta'].sum()

# 4. Sort and Calculate Cumulative Sum (State Value)
events = events.sort_values(['source_ticker', 'target_ticker', 'date'])
events['Val'] = events.groupby(['source_ticker', 'target_ticker'])['delta'].cumsum()

# 5. Determine Interval End Dates
# The end of the current valid period is the day before the NEXT event date
events['next_date'] = events.groupby(['source_ticker', 'target_ticker'])['date'].shift(-1)
events['end_'] = events['next_date'] - pd.Timedelta(days=1)

# 6. Filter for valid intervals (where Val > 0 and we have a closing date)
final_df = events.dropna(subset=['end_'])
final_df = final_df[final_df['Val'] != 0]

# 7. Final Formatting
final_df = final_df.rename(columns={'date': 'start_'})
final_df = final_df[['source_ticker', 'target_ticker', 'start_', 'end_', 'Val']]

In [ ]:
final_df['Val'].value_counts()

,count
Val,
3,17865
4,14914
2,10472
6,2706
7,1646
5,576
8,453
10,309
11,162


In [ ]:
final_df.to_feather(f'{folder_path}econ_link_filt2.feather')

In [ ]:
final_df['start_'].min()

Timestamp('2003-04-03 00:00:00')

In [4]:
final_df = pd.read_feather(f'{folder_path}econ_link_filt2.feather')

In [5]:
ratios = ratios_info.drop(columns=['gvkey','cusip','adate','qdate'])

In [6]:
ratios.columns

Index(['permno', 'public_date', 'bm', 'pe_inc', 'ps', 'npm', 'roa', 'roe',
       'lt_debt', 'ocf_lct', 'fcf_ocf', 'debt_assets', 'intcov_ratio',
       'cash_ratio', 'quick_ratio', 'curr_ratio', 'inv_turn', 'at_turn',
       'rect_turn', 'pay_turn', 'rd_sale', 'ptb', 'TICKER'],
      dtype='object')

In [7]:
ratios

,permno,public_date,bm,pe_inc,ps,npm,roa,roe,lt_debt,ocf_lct,...,cash_ratio,quick_ratio,curr_ratio,inv_turn,at_turn,rect_turn,pay_turn,rd_sale,ptb,TICKER
0,54594,2010-01-31,1.077,16.550,0.663,0.041,0.095,0.079,0.541,0.491,...,0.466,1.534,3.586,2.272,1.001,6.346,11.295,0.0,1.152,AIR
1,54594,2010-02-28,1.077,16.200,0.649,0.041,0.095,0.079,0.541,0.491,...,0.466,1.534,3.586,2.272,1.001,6.346,11.295,0.0,1.128,AIR
2,54594,2010-03-31,1.077,17.729,0.712,0.041,0.095,0.079,0.541,0.491,...,0.466,1.534,3.586,2.272,1.001,6.346,11.295,0.0,1.237,AIR
3,54594,2010-04-30,0.898,19.821,0.719,0.037,0.086,0.067,0.535,0.794,...,0.545,1.701,3.893,2.312,0.987,6.314,10.717,0.0,1.197,AIR
4,54594,2010-05-31,0.898,16.016,0.581,0.037,0.086,0.067,0.535,0.794,...,0.545,1.701,3.893,2.312,0.987,6.314,10.717,0.0,0.968,AIR
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
665278,15642,2024-08-31,0.340,-0.886,NaN,NaN,-1.022,-1.137,0.028,-5.927,...,5.389,5.651,5.651,NaN,NaN,NaN,0.000,0.0,2.724,INDP
665279,15642,2024-09-30,0.340,-0.667,NaN,NaN,-1.022,-1.137,0.028,-5.927,...,5.389,5.651,5.651,NaN,NaN,NaN,0.000,0.0,2.051,INDP
665280,15642,2024-10-31,0.340,-0.629,NaN,NaN,-1.022,-1.137,0.028,-5.927,...,5.389,5.651,5.651,NaN,NaN,NaN,0.000,0.0,1.935,INDP
665281,15642,2024-11-30,0.475,-0.600,NaN,NaN,-1.187,-1.387,0.018,-5.443,...,4.460,4.651,4.651,NaN,NaN,NaN,0.000,0.0,2.371,INDP


In [10]:
sector_info['YEAR'] = pd.to_datetime(sector_info['datadate']).dt.year
sector_info['MONTH'] = pd.to_datetime(sector_info['datadate']).dt.month

In [14]:
sector_info3 = sector_info[['tic','gsector','YEAR','MONTH']].drop_duplicates().dropna()

In [16]:
from scipy.cluster.hierarchy import linkage, fcluster
from sklearn.cluster import HDBSCAN

ratios = ratios.rename(columns={"public_date": "DATE"})
ratios["DATETIME"] = pd.to_datetime(ratios["DATE"], format="%Y-%m-%d")
ratios["YEAR"] = ratios["DATETIME"].dt.year
ratios["MONTH"] = ratios["DATETIME"].dt.month

In [19]:
ratios_sec = ratios.merge(sector_info3, left_on=['TICKER','YEAR','MONTH'], right_on=['tic','YEAR','MONTH'], how='left')
ratios_sec = ratios_sec[ratios_sec['TICKER'].isin(sp500['ticker'])]

In [21]:
ratios_sec = ratios_sec[ratios_sec['tic'].notna()]

In [24]:
ratios_col = [
    "bm", "pe_inc", "ps", "npm", "roa", "roe",
    "lt_debt", "ocf_lct", "fcf_ocf", "debt_assets",
    "intcov_ratio", "cash_ratio", "quick_ratio", "curr_ratio",
    "inv_turn", "at_turn", "rect_turn", "pay_turn", "rd_sale", "ptb"
]

In [25]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.impute import KNNImputer

pipe = Pipeline([
    ("scale", StandardScaler()),
    ("knn", KNNImputer(n_neighbors=5)),
])

def knn_fill(group_df):
    group_df = group_df.copy()
    arr = group_df[ratios_col].to_numpy()
    filled = pipe.fit_transform(arr)
    group_df[ratios_col] = filled
    return group_df

In [31]:
uniq_tickers = list(set(final_df['source_ticker']).union(final_df['target_ticker']))

In [33]:
from sklearn.linear_model import LinearRegression

# Global configuration
factor_etf_lst = ["IWM", "SIZE", "MTUM", "VLUE", "QUAL"]

def fit_lr_model(rets_pivot, x_labels, y_labels):
    """
    Fits the LR model using pre-pivoted data to save time.
    """
    # Ensure columns exist
    valid_x = [x for x in x_labels if x in rets_pivot.columns]
    valid_y = [y for y in y_labels if y in rets_pivot.columns]

    if not valid_x or not valid_y:
        return None, valid_y

    X = rets_pivot[valid_x].to_numpy()
    y = rets_pivot[valid_y].to_numpy()

    # Logic: Drop row if ANY X or ANY y is NaN (matching Polars strict mask)
    # Note: This is aggressive. If one stock in y is NaN, the row is dropped for fitting.
    mask_x = np.isnan(X).any(axis=1)
    mask_y = np.isnan(y).any(axis=1)
    valid_mask = ~(mask_x | mask_y)

    if valid_mask.sum() == 0:
        return None, valid_y

    masked_X = X[valid_mask, :]
    masked_y = y[valid_mask, :]

    model = LinearRegression().fit(masked_X, masked_y)
    return model, valid_y

def get_resid_pandas(rets_pivot, x_labels, y_labels, model):
    """
    Calculates residuals based on the fitted model.
    """
    # Align Data
    valid_x = [x for x in x_labels if x in rets_pivot.columns]
    # We must use the exact same y_labels used during fitting
    valid_y = [y for y in y_labels if y in rets_pivot.columns]

    X = rets_pivot[valid_x].to_numpy()
    y = rets_pivot[valid_y].to_numpy()

    # Handle NaNs in X for prediction (Polars code set them to 0 temporarily)
    X_filled = np.nan_to_num(X, nan=0.0)

    # Predict
    y_pred = model.predict(X_filled)
    resid = y - y_pred

    # Apply NaN mask back (if X was NaN, result should be NaN)
    nan_mask = np.isnan(X).any(axis=1)
    resid[nan_mask, :] = np.nan

    return resid, valid_y

In [ ]:
# ---------------------------------------------------------
# PART 1: Generate Residuals
# ---------------------------------------------------------
lookback_resid_list = []
forward_resid_list = []

# Ensure Date types
rets['DATETIME'] = pd.to_datetime(rets['DATETIME'])

for window in trading_window.itertuples(index=False):
    yr, month, trading_start_date, trading_end_date, lookback_start_date, lookback_end_date, _, prev_month_end_date = window

    # 1. Get Universe for this window
    current_universe = sector_cluster_universe[
        (sector_cluster_universe["YEAR"] == prev_month_end_date.year) &
        (sector_cluster_universe["MONTH"] == prev_month_end_date.month)
    ]
    stock_lst = current_universe["TICKER"].tolist()
    full_stock_lst = factor_etf_lst + stock_lst

    # 2. Filter Returns (Optimization: Filter date once, pivot once)
    mask_full = (
        (rets["DATETIME"] >= lookback_start_date) &
        (rets["DATETIME"] <= trading_end_date) &
        (rets["TICKER"].isin(full_stock_lst))
    )
    rets_window = rets.loc[mask_full].copy()

    # Pivot Wide (Index=Datetime, Cols=Ticker)
    rets_pivot = rets_window.pivot_table(index="DATETIME", columns="TICKER", values="RET")

    # Split into Lookback and Trading for Model Fitting
    # Using index slicing on the pivoted dataframe
    pivot_lookback = rets_pivot.loc[lookback_start_date:lookback_end_date]

    # 3. Fit Model
    model, valid_y_cols = fit_lr_model(pivot_lookback, factor_etf_lst, stock_lst)

    if model is None:
        continue

    # 4. Get Residuals for the whole window
    resid_matrix, fitted_stocks = get_resid_pandas(rets_pivot, factor_etf_lst, valid_y_cols, model)

    # 5. Convert Matrix back to Long DataFrame
    resid_df = pd.DataFrame(resid_matrix, index=rets_pivot.index, columns=fitted_stocks)
    resid_melted = resid_df.reset_index().melt(id_vars="DATETIME", var_name="TICKER", value_name="RESID")

    # Add Metadata (gsector, YEAR, MONTH)
    # Join with rets_window to get the original YEAR/MONTH/gsector columns efficiently
    meta_cols = rets_window[['TICKER', 'DATETIME', 'gsector', 'YEAR', 'MONTH']].drop_duplicates(['TICKER', 'DATETIME'])
    resid_final = pd.merge(resid_melted, meta_cols, on=['TICKER', 'DATETIME'], how='left')

    # 6. Split and Append
    lb_mask = (resid_final["DATETIME"] >= lookback_start_date) & (resid_final["DATETIME"] <= lookback_end_date)
    fw_mask = (resid_final["DATETIME"] >= trading_start_date) & (resid_final["DATETIME"] <= trading_end_date)

    lb_df = resid_final[lb_mask].copy()
    lb_df["LOOKBACK_START_DATE"] = lookback_start_date
    lb_df["LOOKBACK_END_DATE"] = lookback_end_date

    fw_df = resid_final[fw_mask].copy()
    fw_df["TRADING_START_DATE"] = trading_start_date
    fw_df["TRADING_END_DATE"] = trading_end_date

    lookback_resid_list.append(lb_df)
    forward_resid_list.append(fw_df)

# Concatenate Results
lookback_resid = pd.concat(lookback_resid_list, ignore_index=True)
forward_resid = pd.concat(forward_resid_list, ignore_index=True)

# Select final columns
lb_cols = ["TICKER", "gsector", "LOOKBACK_START_DATE", "LOOKBACK_END_DATE", "DATETIME", "YEAR", "MONTH", "RESID"]
fw_cols = ["TICKER", "gsector", "TRADING_START_DATE", "TRADING_END_DATE", "DATETIME", "YEAR", "MONTH", "RESID"]

lookback_resid = lookback_resid[lb_cols]
forward_resid = forward_resid[fw_cols]

# Save
lookback_resid.to_feather(f"{folder_path}lookback_resid.feather")
forward_resid.to_feather(f"{folder_path}forward_resid.feather")

In [ ]:



# ---------------------------------------------------------
# PART 2: Calculate Correlations (TIC)
# ---------------------------------------------------------

def get_spearman_matrix(arr):
    """
    Calculates Spearman correlation matrix on a numpy array.
    Input: Array (T time steps, N assets)
    """
    # Rank data column-wise (axis=0)
    # pandas rank is safer for NaNs, but for speed on numpy:
    # We assume 'arr' has already been cleaned of rows with NaNs
    ranks = np.apply_along_axis(lambda x: np.argsort(np.argsort(x)), 0, arr)
    return np.corrcoef(ranks, rowvar=False)

def get_cluster_rho(wide_df, ticker_lst, yr, month):
    # Select columns and drop rows with ANY NaN (Standard Spearman requirement)
    # Ensure we only pick tickers that exist in the wide_df
    valid_tickers = [t for t in ticker_lst if t in wide_df.columns]

    if len(valid_tickers) < 2:
        return []

    data = wide_df[valid_tickers].dropna().to_numpy()

    if data.shape[0] < 2: # Need at least 2 time points
        return []

    # Calculate Matrix
    rho = get_spearman_matrix(data)

    # Extract Upper Triangle
    idx_i, idx_j = np.triu_indices_from(rho, k=1)

    # Prepare result list
    res = []
    names_i = np.array(valid_tickers)[idx_i]
    names_j = np.array(valid_tickers)[idx_j]
    corrs = rho[idx_i, idx_j]

    for t1, t2, val in zip(names_i, names_j, corrs):
        res.append((yr, month, t1, t2, val))

    return res

lookback_corr_list = []
forward_corr_list = []

for window in trading_window.itertuples(index=False):
    yr, month, trading_start_date, trading_end_date, lookback_start_date, lookback_end_date, _, prev_month_end_date = window

    # 1. Define Clusters
    cluster_df = sector_cluster_universe[
        sector_cluster_universe["DATETIME"] == prev_month_end_date
    ]

    # Group tickers by cluster
    clusters = cluster_df.groupby(["gsector", "dbscan_cluster_grp"])["TICKER"].apply(list).reset_index()

    # 2. Pivot Residuals for this window
    # Lookback
    lb_window = lookback_resid[
        (lookback_resid["LOOKBACK_START_DATE"] == lookback_start_date) &
        (lookback_resid["LOOKBACK_END_DATE"] == lookback_end_date)
    ]
    lb_wide = lb_window.pivot_table(index="DATETIME", columns="TICKER", values="RESID")

    # Forward
    fw_window = forward_resid[
        (forward_resid["TRADING_START_DATE"] == trading_start_date) &
        (forward_resid["TRADING_END_DATE"] == trading_end_date)
    ]
    fw_wide = fw_window.pivot_table(index="DATETIME", columns="TICKER", values="RESID")

    # 3. Iterate Clusters and Calculate Rho
    for row in clusters.itertuples(index=False):
        ticker_lst = row.TICKER

        # Lookback
        lb_res = get_cluster_rho(lb_wide, ticker_lst, yr, month)
        lookback_corr_list.extend(lb_res)

        # Forward
        fw_res = get_cluster_rho(fw_wide, ticker_lst, yr, month)
        forward_corr_list.extend(fw_res)

# Create Final DataFrames
cols = ["YEAR", "MONTH", "TICKER_1", "TICKER_2", "SPEARMAN_RHO"]

lookback_correlation_pairs = pd.DataFrame(lookback_corr_list, columns=cols)
forward_correlation_pairs = pd.DataFrame(forward_corr_list, columns=cols)

# Type casting
type_map = {"YEAR": "int32", "MONTH": "int8", "SPEARMAN_RHO": "float64"}
lookback_correlation_pairs = lookback_correlation_pairs.astype(type_map)
forward_correlation_pairs = forward_correlation_pairs.astype(type_map)

# Save
lookback_correlation_pairs.to_parquet("./data/lookback_correlation_pairs.parquet")
# forward_correlation_pairs.to_parquet(...)